In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
#read COVID19 data by County
csv_file1 = "data/covid19_usafacts.csv"
usacovidfacts_data_all_df = pd.read_csv(csv_file1)
usacovidfacts_data_all_df.head()

,county_fips_code,county_name,state,state_fips_code,date,confirmed_cases,deaths
0,0,Statewide Unallocated,AL,1,5/1/2020,0,0
1,0,Statewide Unallocated,AL,1,5/2/2020,0,0
2,0,Statewide Unallocated,AL,1,5/3/2020,0,0
3,0,Statewide Unallocated,AL,1,5/4/2020,0,0
4,0,Statewide Unallocated,AL,1,5/5/2020,0,0


In [3]:
#rename columns in the dataframe to match the database schema
usacovidfacts_data_df = usacovidfacts_data_all_df[['county_fips_code','county_name', 'state', 'date', 'confirmed_cases', 'deaths']].copy()
usacovidfacts_data_df = usacovidfacts_data_df.rename(columns={"county_fips_code":"countyfipscode","state":"stateabr", "county_name":"countyname", "date":"date", "confirmed_cases":"confirmedcases", "deaths":"deaths"})
usacovidfacts_data_df.head()

,countyfipscode,countyname,stateabr,date,confirmedcases,deaths
0,0,Statewide Unallocated,AL,5/1/2020,0,0
1,0,Statewide Unallocated,AL,5/2/2020,0,0
2,0,Statewide Unallocated,AL,5/3/2020,0,0
3,0,Statewide Unallocated,AL,5/4/2020,0,0
4,0,Statewide Unallocated,AL,5/5/2020,0,0


In [4]:
#read US population data by County
csv_file1 = "data/covid_county_population_usafacts.csv"
usapopulationfacts_data_all_df = pd.read_csv(csv_file1)
usapopulationfacts_data_all_df.head()

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869
2,1003,Baldwin County,AL,223234
3,1005,Barbour County,AL,24686
4,1007,Bibb County,AL,22394


In [5]:
#rename columns in the dataframe to match the database schema
usapopulationfacts_data_df = usapopulationfacts_data_all_df[['countyFIPS','County Name', 'State', 'population']].copy()
usapopulationfacts_data_df = usapopulationfacts_data_df.rename(columns={"countyFIPS":"countyfipscode", "County Name": "countyname", "State":"stateabr", "population":"population"})
usapopulationfacts_data_df.head()

,countyfipscode,countyname,stateabr,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869
2,1003,Baldwin County,AL,223234
3,1005,Barbour County,AL,24686
4,1007,Bibb County,AL,22394


In [6]:
# Get State related data by webscraping the below URL.  This data will provide state full name for abbrevations in 
# USA Fact data
url = "https://developers.google.com/public-data/docs/canonical/states_csv"
tables = pd.read_html(url)
tables

[   state   latitude   longitude                  name
 0     AK  63.588753 -154.493062                Alaska
 1     AL  32.318231  -86.902298               Alabama
 2     AR  35.201050  -91.831833              Arkansas
 3     AZ  34.048928 -111.093731               Arizona
 4     CA  36.778261 -119.417932            California
 5     CO  39.550051 -105.782067              Colorado
 6     CT  41.603221  -73.087749           Connecticut
 7     DC  38.905985  -77.033418  District of Columbia
 8     DE  38.910832  -75.527670              Delaware
 9     FL  27.664827  -81.515754               Florida
 10    GA  32.157435  -82.907123               Georgia
 11    HI  19.898682 -155.665857                Hawaii
 12    IA  41.878003  -93.097702                  Iowa
 13    ID  44.068202 -114.742041                 Idaho
 14    IL  40.633125  -89.398528              Illinois
 15    IN  40.551217  -85.602364               Indiana
 16    KS  39.011902  -98.484246                Kansas
 17    KY 

In [7]:
# take the webscrapped data from table and rename the columns to match the database schema
state_info = tables[0]
state_info.columns = ['stateabr', 'latitude','longitude', 'statename']
state_info.head()

,stateabr,latitude,longitude,statename
0,AK,63.588753,-154.493062,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AZ,34.048928,-111.093731,Arizona
4,CA,36.778261,-119.417932,California


In [8]:
# Create Engine and connection to Database
rds_connection_string = "postgres:[password]@localhost:5432/Covid_data"
engine = create_engine(f'postgresql://{rds_connection_string}')
conn = engine.connect()

In [9]:
#display all the tables in the database
engine.table_names()

['statedata', 'covid19data', 'countydata']

In [10]:
#insert state data into statedata table
state_info.to_sql(name='statedata', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from statedata', con=engine).head()

,stateabr,latitude,longitude,statename
0,AK,63.588753,-154.493062,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AZ,34.048928,-111.093731,Arizona
4,CA,36.778261,-119.417932,California


In [11]:
#insert covid facts by county into covid19data table
usacovidfacts_data_df.to_sql(name='covid19data', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from covid19data', con=engine).head()

,countyfipscode,countyname,stateabr,date,confirmedcases,deaths
0,0,Statewide Unallocated,AL,5/1/2020,0,0
1,0,Statewide Unallocated,AL,5/2/2020,0,0
2,0,Statewide Unallocated,AL,5/3/2020,0,0
3,0,Statewide Unallocated,AL,5/4/2020,0,0
4,0,Statewide Unallocated,AL,5/5/2020,0,0


In [12]:
#insert usa population by county data into countydata table
usapopulationfacts_data_df.to_sql(name='countydata', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from countydata', con=engine).head()

,countyfipscode,countyname,stateabr,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869
2,1003,Baldwin County,AL,223234
3,1005,Barbour County,AL,24686
4,1007,Bibb County,AL,22394


In [13]:
# Get county population, covid confirmed cases, and covid deaths data for seven days [5/1/2020 to 5/7/2020]
# by joining all three tables while filtering out rows for any counties that have population reported as 0

querystring = (
    "select "
    + "  statedata.statename, statedata.stateabr, countydata.countyname, " 
    + "  countydata.population, covid19data.date, covid19data.confirmedcases, "
    + "  covid19data.deaths "
    + "from "
    + "  covid19data, countydata, statedata "
    + "where "
    + "  covid19data.countyfipscode = countydata.countyfipscode and"
    + "  statedata.stateabr = countydata.stateabr and"
    + "  countydata.population != 0 "
    + "order by "
    + "  statedata.statename, countydata.countyname, covid19data.date"
)

pd.read_sql_query(querystring, con=engine)

,statename,stateabr,countyname,population,date,confirmedcases,deaths
0,Alabama,AL,Autauga County,55869,5/1/2020,42,3
1,Alabama,AL,Autauga County,55869,5/2/2020,45,3
2,Alabama,AL,Autauga County,55869,5/3/2020,48,3
3,Alabama,AL,Autauga County,55869,5/4/2020,53,3
4,Alabama,AL,Autauga County,55869,5/5/2020,53,3
...,...,...,...,...,...,...,...
21989,Wyoming,WY,Weston County,6927,5/3/2020,0,0
21990,Wyoming,WY,Weston County,6927,5/4/2020,0,0
21991,Wyoming,WY,Weston County,6927,5/5/2020,0,0
21992,Wyoming,WY,Weston County,6927,5/6/2020,0,0


In [14]:
# Get aggregated population, confirmed cases and death total at state level
# by joining all three tables

querystring = (
    "select "
    + "  statedata.statename, statedata.stateabr, " 
    + "  sum(countydata.population) as population, "
    + "  sum(covid19data.confirmedcases) as confirmedcases, "
    + "  sum(covid19data.deaths) as deaths "
    + "from "
    + "  covid19data, countydata, statedata "
    + "where "
    + "  covid19data.countyfipscode = countydata.countyfipscode and"
    + "  statedata.stateabr = countydata.stateabr and "
    + "  covid19data.date = '5/7/2020'"
    + "group by "
    + "  statedata.statename, statedata.stateabr "
    + "order by "
    + "  statedata.statename"
)

pd.read_sql_query(querystring, con=engine).head(10)

,statename,stateabr,population,confirmedcases,deaths
0,Alabama,AL,4903185.0,16751.0,716.0
1,Alaska,AK,731545.0,8079.0,357.0
2,Arizona,AZ,7278717.0,17649.0,797.0
3,Arkansas,AR,3017804.0,11246.0,435.0
4,California,CA,39512223.0,70007.0,2887.0


In [15]:
# Get aggregated population, confirmed cases and percent of the population infected at state level
# by joining all three tables and sorting the data by showing highest infected percentage on top

querystring = (
    "select "
    + "  statedata.statename, statedata.stateabr, " 
    + "  sum(countydata.population) as population, "
    + "  sum(covid19data.confirmedcases) as confirmedcases, "
    + "  ROUND(((sum(covid19data.confirmedcases) / sum(countydata.population)) * 100),2) as percentinfected "
    + "from "
    + "  covid19data, countydata, statedata "
    + "where "
    + "  covid19data.countyfipscode = countydata.countyfipscode and"
    + "  statedata.stateabr = countydata.stateabr and "
    + "  covid19data.date = '5/7/2020'"
    + "group by "
    + "  statedata.statename, statedata.stateabr "
    + "order by "
    + "  percentinfected desc"
)

pd.read_sql_query(querystring, con=engine).head(10)

,statename,stateabr,population,confirmedcases,percentinfected
0,New York,NY,19453561.0,342879.0,1.76
1,New Jersey,NJ,8882190.0,140788.0,1.59
2,Rhode Island,RI,1059361.0,16224.0,1.53
3,Wyoming,WY,578759.0,8188.0,1.41
4,Delaware,DE,973764.0,13614.0,1.40
5,Vermont,VT,623989.0,8616.0,1.38
6,South Dakota,SD,884659.0,10610.0,1.20
7,North Dakota,ND,762062.0,9076.0,1.19
8,Massachusetts,MA,6892503.0,81098.0,1.18
9,Alaska,AK,731545.0,8079.0,1.10
